In [119]:
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")
import numpy as np

data = [['2018-12-28'], 
        ['2018-12-29'], 
        ['2019-01-04'], 
        ['2019-01-05']]
failed = pd.DataFrame(data, 
                      columns=['fail_date']).astype({'fail_date':'datetime64[ns]'})
data = [['2018-12-30'], 
        ['2018-12-31'], 
        ['2019-01-01'], 
        ['2019-01-02'], 
        ['2019-01-03'], 
        ['2019-01-06']]
succeeded = pd.DataFrame(data, 
                         columns=['success_date']).astype({'success_date':'datetime64[ns]'})

display(failed, succeeded)

range_start = pd.Timestamp("2019-01-01")
range_end = pd.Timestamp("2019-12-31")

,fail_date
0,2018-12-28
1,2018-12-29
2,2019-01-04
3,2019-01-05


,success_date
0,2018-12-30
1,2018-12-31
2,2019-01-01
3,2019-01-02
4,2019-01-03
5,2019-01-06


In [120]:
failed['state'] = 'failed'
failed = failed.rename(columns={'fail_date': 'date'})
display(failed)

,date,state
0,2018-12-28,failed
1,2018-12-29,failed
2,2019-01-04,failed
3,2019-01-05,failed


In [121]:
succeeded['state'] = 'succeeded'
succeeded = succeeded.rename(columns={'success_date': 'date'})
display(succeeded)

,date,state
0,2018-12-30,succeeded
1,2018-12-31,succeeded
2,2019-01-01,succeeded
3,2019-01-02,succeeded
4,2019-01-03,succeeded
5,2019-01-06,succeeded


In [122]:
df = pd.concat([failed, succeeded])
df = df.sort_values(by='date', ascending=True)
display(df)

,date,state
0,2018-12-28,failed
1,2018-12-29,failed
0,2018-12-30,succeeded
1,2018-12-31,succeeded
2,2019-01-01,succeeded
3,2019-01-02,succeeded
4,2019-01-03,succeeded
2,2019-01-04,failed
3,2019-01-05,failed
5,2019-01-06,succeeded


In [123]:
df = df[df['date'].between('2019-01-01', '2019-12-31')]
display(df)

,date,state
2,2019-01-01,succeeded
3,2019-01-02,succeeded
4,2019-01-03,succeeded
2,2019-01-04,failed
3,2019-01-05,failed
5,2019-01-06,succeeded


In [124]:
# df['period'] = (df['state'] != df['state'].shift()).cumsum()

df["period_state_previous"] = df["state"].shift(periods = 1)
df["period_switch"] = np.where(df["state"] != df["period_state_previous"], 1, 0)
df["period"] = df["period_switch"].cumsum()

display(df)

,date,state,period_state_previous,period_switch,period
2,2019-01-01,succeeded,None,1,1
3,2019-01-02,succeeded,succeeded,0,1
4,2019-01-03,succeeded,succeeded,0,1
2,2019-01-04,failed,succeeded,1,2
3,2019-01-05,failed,failed,0,2
5,2019-01-06,succeeded,failed,1,3


In [125]:
df = df.groupby(['period', 
                 'state']).agg(start_date=('date', 'min'), 
                               end_date=('date', 'max'))
df = df.reset_index().rename(columns={'state': 'period_state'})
display(df)

,period,period_state,start_date,end_date
0,1,succeeded,2019-01-01,2019-01-03
1,2,failed,2019-01-04,2019-01-05
2,3,succeeded,2019-01-06,2019-01-06


In [126]:
df = df[['period_state', 'start_date', 'end_date']]
display(df)

,period_state,start_date,end_date
0,succeeded,2019-01-01,2019-01-03
1,failed,2019-01-04,2019-01-05
2,succeeded,2019-01-06,2019-01-06


In [127]:
# failed['state'] = 'failed'
# failed = failed.rename(columns={'fail_date': 'date'})
# succeeded['state'] = 'succeeded'
# succeeded = succeeded.rename(columns={'success_date': 'date'})
# df = pd.concat([failed, succeeded])
# df = df.sort_values(by='date', ascending=True)
# df = df[(df['date'] >= '2019-01-01') & (df['date'] <= '2019-12-31')]
# df['period'] = (df['state'] != df['state'].shift()).cumsum()
# df = df.groupby(['period', 
#                  'state']).agg(start_date=('date', 'min'), 
#                                end_date=('date', 'max'))
# df = df.reset_index().rename(columns={'state': 'period_state'})
# df = df[['period_state', 'start_date', 'end_date']]